# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**: Juan Bernardo Orozco Quirarte

**Professor**: Pablo Camarillo Ramirez

In [13]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://7d7f1cd1d5d2:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

### Agencies

In [14]:
# Build schema
# Import your module
from lib.bernardoorozco.spark_utils import SparkUtils

schema_agencies = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])

print("Agencies")
df_agencies = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/agencies")
df_agencies.show(truncate=False)


Agencies
+---------+-----------------------------------------------------+
|agency_id|agency_info                                          |
+---------+-----------------------------------------------------+
|1        |{'agency_name': 'NYC Rentals', 'city': 'New York'}   |
|2        |{'agency_name': 'LA Car Rental', 'city': 'Londres'}  |
|3        |{'agency_name': 'Zapopan Auto', 'city': 'Zapopan'}   |
|4        |{'agency_name': 'SF Cars', 'city': 'San Francisco'}  |
|5        |{'agency_name': 'Mexico Cars', 'city': 'Mexico City'}|
+---------+-----------------------------------------------------+



In [15]:
from pyspark.sql.functions import from_json
from pyspark.sql.functions import get_json_object


df_agencies = df_agencies.withColumn("agency_name", get_json_object("agency_info", "$.agency_name"))
df_agencies.show(truncate=False)

+---------+-----------------------------------------------------+-------------+
|agency_id|agency_info                                          |agency_name  |
+---------+-----------------------------------------------------+-------------+
|1        |{'agency_name': 'NYC Rentals', 'city': 'New York'}   |NYC Rentals  |
|2        |{'agency_name': 'LA Car Rental', 'city': 'Londres'}  |LA Car Rental|
|3        |{'agency_name': 'Zapopan Auto', 'city': 'Zapopan'}   |Zapopan Auto |
|4        |{'agency_name': 'SF Cars', 'city': 'San Francisco'}  |SF Cars      |
|5        |{'agency_name': 'Mexico Cars', 'city': 'Mexico City'}|Mexico Cars  |
+---------+-----------------------------------------------------+-------------+



### Brands

In [16]:
schema_brands = SparkUtils.generate_schema([("brand_id", "int"), ("brand_info", "string")])

print("Brands")
df_brands = spark.read.schema(schema_brands).option("header", True).csv("/opt/spark/work-dir/data/car_service/brands")
df_brands.show(truncate=False)

Brands
+--------+-----------------------------------------------------+
|brand_id|brand_info                                           |
+--------+-----------------------------------------------------+
|1       |{'brand_name': 'Mercedes-Benz', 'country': 'Germany'}|
|2       |{'brand_name': 'BMW', 'country': 'Germany'}          |
|3       |{'brand_name': 'Audi', 'country': 'Germany'}         |
|4       |{'brand_name': 'Ford', 'country': 'US'}              |
|5       |{'brand_name': 'BYD', 'country': 'China'}            |
|6       |{'brand_name': 'Honda', 'country': 'Japan'}          |
|7       |{'brand_name': 'Toyota', 'country': 'Japan'}         |
+--------+-----------------------------------------------------+



In [17]:
df_brands = df_brands.withColumn("brand_name", get_json_object("brand_info", "$.brand_name"))

### Cars

In [18]:
schema_cars = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])

print("Cars")
df_cars = spark.read.schema(schema_cars).option("header", True).csv("/opt/spark/work-dir/data/car_service/cars")
df_cars.show(truncate=False)

Cars
+------+----------------------------------------------------------------------------------------+
|car_id|car_info                                                                                |
+------+----------------------------------------------------------------------------------------+
|1     |{'car_name': 'Chang-Fisher Model 7', 'brand_id': 5, 'price_per_day': 139}               |
|2     |{'car_name': 'Sheppard-Tucker Model 4', 'brand_id': 6, 'price_per_day': 70}             |
|3     |{'car_name': 'Faulkner-Howard Model 5', 'brand_id': 3, 'price_per_day': 53}             |
|4     |{'car_name': 'Wagner LLC Model 1', 'brand_id': 5, 'price_per_day': 89}                  |
|5     |{'car_name': 'Campos PLC Model 4', 'brand_id': 4, 'price_per_day': 112}                 |
|6     |{'car_name': 'Archer-Patel Model 2', 'brand_id': 3, 'price_per_day': 55}                |
|7     |{'car_name': 'Patrick, Barrera and Collins Model 6', 'brand_id': 6, 'price_per_day': 66}|
|8     |{'car_n

In [19]:
df_cars = (
    df_cars
    .withColumn("car_name", get_json_object("car_info", "$.car_name"))
    .withColumn("brand_id", get_json_object("car_info", "$.brand_id").cast("int"))
    .withColumn("price_per_day", get_json_object("car_info", "$.price_per_day").cast("int"))
)

### Customers

In [20]:
schema_customers = SparkUtils.generate_schema([("customer_id", "int"), ("customer_info", "string")])

print("Customers")
df_customers = spark.read.schema(schema_customers).option("header", True).csv("/opt/spark/work-dir/data/car_service/customers")
df_customers.show(truncate=False)

Customers
+-----------+-------------------------------------------------------------------------+
|customer_id|customer_info                                                            |
+-----------+-------------------------------------------------------------------------+
|1          |{'customer_name': 'Tiffany Riley', 'city': 'Monterrey', 'age': 32}       |
|2          |{'customer_name': 'Matthew Davies', 'city': 'Monterrey', 'age': 36}      |
|3          |{'customer_name': 'Rebecca Miller', 'city': 'Mexico City', 'age': 30}    |
|4          |{'customer_name': 'Katelyn Mccoy', 'city': 'New York', 'age': 34}        |
|5          |{'customer_name': 'Dana Dennis', 'city': 'Zapopan', 'age': 26}           |
|6          |{'customer_name': 'Daniel Norton', 'city': 'Mexico City', 'age': 34}     |
|7          |{'customer_name': 'Robert Garcia', 'city': 'Zapopan', 'age': 47}         |
|8          |{'customer_name': 'Michael Williams', 'city': 'Monterrey', 'age': 33}    |
|9          |{'custome

In [21]:
df_customers = df_customers.withColumn("customer_name", get_json_object("customer_info", "$.customer_name"))

### Rentals

In [22]:
schema_rentals = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])

print("Rentals")
df_rentals = spark.read.schema(schema_rentals).option("header", True).csv("/opt/spark/work-dir/data/car_service/rentals")
df_rentals.show(truncate=False)

Rentals
+---------+--------------------------------------------------+
|rental_id|rental_info                                       |
+---------+--------------------------------------------------+
|11891    |{'car_id': 21, 'customer_id': 71, 'agency_id': 1} |
|11892    |{'car_id': 11, 'customer_id': 52, 'agency_id': 2} |
|11893    |{'car_id': 22, 'customer_id': 116, 'agency_id': 4}|
|11894    |{'car_id': 5, 'customer_id': 107, 'agency_id': 1} |
|11895    |{'car_id': 4, 'customer_id': 53, 'agency_id': 4}  |
|11896    |{'car_id': 8, 'customer_id': 131, 'agency_id': 2} |
|11897    |{'car_id': 23, 'customer_id': 66, 'agency_id': 3} |
|11898    |{'car_id': 24, 'customer_id': 60, 'agency_id': 4} |
|11899    |{'car_id': 27, 'customer_id': 92, 'agency_id': 2} |
|11900    |{'car_id': 3, 'customer_id': 40, 'agency_id': 4}  |
|11901    |{'car_id': 3, 'customer_id': 35, 'agency_id': 1}  |
|11902    |{'car_id': 3, 'customer_id': 72, 'agency_id': 1}  |
|11903    |{'car_id': 29, 'customer_id': 58, 'a

In [23]:
df_rentals = (
    df_rentals.withColumn("car_id", get_json_object("rental_info", "$.car_id").cast("int")).withColumn("agency_id", get_json_object("rental_info", "$.agency_id").cast("int")).withColumn("customer_id", get_json_object("rental_info", "$.customer_id").cast("int"))
)

### Join

In [24]:
df_final = (
    df_rentals
    .join(df_cars, "car_id", "inner")
    .join(df_brands, "brand_id", "inner")
    .join(df_agencies, "agency_id", "inner")
    .join(df_customers, "customer_id", "inner")
    .select("rental_id","car_name","brand_name","agency_name","customer_name")
)
df_final.show(truncate=False)

+---------+-----------------------------------+-------------+-------------+---------------+
|rental_id|car_name                           |brand_name   |agency_name  |customer_name  |
+---------+-----------------------------------+-------------+-------------+---------------+
|11891    |Wallace-Carlson Model 9            |Honda        |NYC Rentals  |Margaret Jones |
|11892    |Grimes-Green Model 8               |Audi         |LA Car Rental|Albert Williams|
|11893    |Stewart-Allen Model 5              |Honda        |SF Cars      |Caleb Fleming  |
|11894    |Campos PLC Model 4                 |Ford         |NYC Rentals  |Andrew Butler  |
|11895    |Wagner LLC Model 1                 |BYD          |SF Cars      |Kristin Potts  |
|11896    |Jones, Jefferson and Rivera Model 7|BMW          |LA Car Rental|Jeremy Parks   |
|11897    |Lopez and Sons Model 9             |BMW          |Zapopan Auto |Terry Wells    |
|11898    |Salazar Ltd Model 8                |Mercedes-Benz|SF Cars      |Marc 

In [26]:
sc.stop()